## Z Scaling

In [159]:
X = final_df['r1g1']
from sklearn.preprocessing import scale
r1g1_scale = scale(X)
np.mean(r1g1_scale), np.std(r1g1_scale)

(1.5391861644074873e-18, 1.0)

In [160]:
for i in final_df.columns[15:24].tolist()+final_df.columns[25:].tolist():
    final_df[i] = scale(final_df[i])
    #np.mean(final_df[i]), np.std(final_df[i])

## PCA

In [161]:
pdf = final_df[final_df.columns[15:24].tolist()+final_df.columns[25:].tolist()]

In [162]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
pca.fit(pdf)
pca_df = pca.transform(pdf)

In [163]:
pca_df=pd.DataFrame(pca_df)

## Z-scale after PCA

In [164]:
for i in pca_df.columns:
    pca_df[i] = scale(pca_df[i])

## Score 1

In [166]:
pca_df['8']=pca_df.abs().sum(axis=1)

In [167]:
rankpca=pca_df.sort_values(by='8',ascending=True)

In [168]:
rankpca['9']=range(1,1070995)

In [169]:
score = rankpca[['9']]
score.columns=['score1']

## Autoencoder

In [172]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
pca.fit(pdf)
pca_df = pca.transform(pdf)
pca_df=pd.DataFrame(pca_df)
for i in pca_df.columns:
    pca_df[i] = scale(pca_df[i])

In [174]:
import pandas as pd
import numpy as np
from scipy import stats
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import StandardScaler
from pylab import rcParams
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

In [175]:
nb_epoch = 10
batch_size = 128
input_dim = pca_df.shape[1] #num of columns, 8
encoding_dim = 14
hidden_dim = int(encoding_dim / 2) #i.e. 7
learning_rate = 1e-7

In [176]:
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
encoder = Dense(hidden_dim, activation="relu")(encoder)
decoder = Dense(hidden_dim, activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

In [177]:
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

cp = ModelCheckpoint(filepath="autoencoder_fraud.h5",
                               save_best_only=True,
                               verbose=0)

tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=True)

history = autoencoder.fit(pca_df, pca_df,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(pca_df, pca_df),
                    verbose=1,
                    callbacks=[cp, tb]).history

Train on 1070994 samples, validate on 1070994 samples
Epoch 1/10
1070994/1070994 [==============================] - 16s 15us/step - loss: 0.9754 - acc: 0.7772 - val_loss: 0.9674 - val_acc: 0.8194
Epoch 2/10
1070994/1070994 [==============================] - 15s 14us/step - loss: 0.9622 - acc: 0.8512 - val_loss: 0.9518 - val_acc: 0.8741
Epoch 3/10
1070994/1070994 [==============================] - 15s 14us/step - loss: 0.9443 - acc: 0.8761 - val_loss: 0.9353 - val_acc: 0.8885
Epoch 4/10
1070994/1070994 [==============================] - 15s 14us/step - loss: 0.9297 - acc: 0.8816 - val_loss: 0.9230 - val_acc: 0.8865
Epoch 5/10
1070994/1070994 [==============================] - 15s 14us/step - loss: 0.9193 - acc: 0.8784 - val_loss: 0.9146 - val_acc: 0.8167
Epoch 6/10
1070994/1070994 [==============================] - 15s 14us/step - loss: 0.9097 - acc: 0.8655 - val_loss: 0.9052 - val_acc: 0.8752
Epoch 7/10
1070994/1070994 [==============================] - 15s 14us/step - loss: 0.9019 - a

In [178]:
autoencoder = load_model('autoencoder_fraud.h5')

In [179]:
prediction=autoencoder.predict(pca_df)

In [181]:
prediction.shape

(1070994, 8)

In [182]:
diff=prediction-pca_df

In [183]:
diff['8']=diff.abs().sum(axis=1)

In [184]:
rankdiff=diff.sort_values(by='8',ascending=True)

In [185]:
rankdiff['9']=range(1,1070995)

In [186]:
score[2] = rankdiff[['9']]
score.columns=['score1','score2']

In [187]:
score['total']=score['score1']+score['score2']

In [189]:
score.sort_values(by='total',ascending=False).head(20)

,score1,score2,total
632815,1070994,1070994,2141988
565391,1070993,1070993,2141986
1067359,1070992,1070992,2141984
917941,1070991,1070991,2141982
585117,1070990,1070990,2141980
85885,1070989,1070989,2141978
585438,1070988,1070988,2141976
565397,1070987,1070987,2141974
920627,1070986,1070986,2141972
556608,1070985,1070985,2141970


In [149]:
score.to_csv('score.csv')

In [190]:
fdf.iloc[632815]

RECORD                  632816
BBLE                4018420001
B                            4
BLOCK                     1842
LOT                          1
EASEMENT                   NaN
OWNER       864163 REALTY, LLC
BLDGCL                      D9
TAXCLASS                     2
LTFRONT                    157
LTDEPTH                     95
EXT                        NaN
STORIES                      1
FULLVAL               2.93e+06
AVLAND              1.3185e+06
AVTOT               1.3185e+06
EXLAND                       0
EXTOT                        0
EXCD1                      NaN
STADDR          86-55 BROADWAY
ZIP                      11373
EXMPTCL                    NaN
BLDFRONT                     1
BLDDEPTH                     1
AVLAND2             1.2012e+06
AVTOT2              1.2012e+06
EXLAND2                    NaN
EXTOT2                     NaN
EXCD2                      NaN
PERIOD                   FINAL
YEAR                   2010/11
VALTYPE                  AC-TR
ZT      